# **Modelling And Evaluating**

## Objectives
* engineer features for modelling and Evaluating

## Inputs

*  The input data, already cleaned and divided into test/train/validating found in:
    * /workspace/mildew-detection/inputs/mildew_dataset/cherry-leaves 
### **Grayscale**
* Same Data but grayscale
    * /workspace/mildew-detection/inputs/mildew_dataset/gray_images
 
## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* I chose to have one rgb and one grayscale to se wich one that gives the best resault



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Downloading neccessary packages

Packages:

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

---

# Setting up the path for train/test/validation

In [ ]:
my_data_dir = "inputs/mildew_dataset/cherry-leaves"
train_path = my_data_dir + "/" + "train"
test_path = my_data_dir + "/" + "test"
val_path = my_data_dir + "/" + "validation"

my_data_dir_gray = "inputs/mildew_dataset/gray_images"
train_path_gray = my_data_dir_gray + "/" + "train"
test_path_gray = my_data_dir_gray + "/" + "test"
val_path_gray = my_data_dir_gray + "/" + "validation"

Setting the output for images and other relevant data

In [ ]:
version = "v1"
version_two = "v2"
file_path_one = f'output/{version}'
file_path_two = f'output/{version_two}'

# creating output for the rgb images
if "output" in os.listdir(current_dir) and version in os.listdir(current_dir + "/output"):
    print("make a new version")
    pass
else:
    os.makedirs(name=file_path_one)
# creating output for the gray scale images
if "output" in os.listdir(current_dir) and version_two in os.listdir(current_dir + "/output"):
    print("make a new version")
    pass
else:
    os.makedirs(name=file_path_two)

setting the lable variable

* for RGB images

In [ ]:
labels = os.listdir(train_path)
print('Label for the images are', labels)

* for grascale images

In [ ]:
labels_gray = os.listdir(train_path_gray)
print('Label for the images are', labels)

In [ ]:
## Import saved image shape embedding
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"output/{version}/image_shape.pkl")
image_shape

---

Visualisation of the amount of images in the dataset

* Showing the amount of images in Dataset to get a better understanding of the Dataset
    * Showing with a barplot


---

### **Doing this only RGB because its the same dataset and the numbers won't be different**

In [ ]:

df_freq = pd.DataFrame([])
for folder in ['train', 'validation', 'test']:
    for label in labels:
        df_freq = df_freq.append(
            pd.Series(data={'Set': folder,
                            'Label': label,
                            'Frequency': int(len(os.listdir(my_data_dir + '/' + folder + '/' + label)))}
                      ),
            ignore_index=True
        )

        print(
            f"* {folder} - {label}: {len(os.listdir(my_data_dir+'/'+ folder + '/' + label))} images")

print("\n")
sns.set_style("whitegrid")
plt.figure(figsize=(8, 5))
sns.barplot(data=df_freq, x='Set', y='Frequency', hue='Label')
plt.savefig(f'{file_path_one}/labels_distribution.png',
            bbox_inches='tight', dpi=150)
plt.show()

Generating images from current images using image agumantation from Tensorflow
* importing neccessary packages for the task 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
augmented_image_data = ImageDataGenerator(rotation_range=25,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

Augumentation for RGB train set

In [ ]:
batch_size = 25  # Set batch size
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=True
                                                     )

train_set.class_indices

Augumentation for RGB test set

In [ ]:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='binary',
                                                                  shuffle=False
                                                                  )

test_set.class_indices

Augumentation for RGB validation set

In [ ]:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False
                                                                        )

validation_set.class_indices

Plot the augumented Images for Test/train/validation set

* Train set

In [ ]:
for _ in range(3):
    img, label = train_set.next()
    print(img.shape)
    plt.imshow(img[0])
    plt.show()

* Test set

In [ ]:
for _ in range(3):
    img, label = test_set.next()
    print(img.shape) 
    plt.imshow(img[0])
    plt.show()

* Validation set

In [ ]:
for _ in range(3):
    img, label = validation_set.next()
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

## Saving the train set indices in "v1"
   * As an pickle file

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path_one}/class_indices.pkl")

---

## **Creating The Model**

---

Import neccessary packages for ML

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

* Creating the model

In [ ]:
def create_tf_model():
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3, 3),
              input_shape=image_shape, activation='relu', ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3),
              input_shape=image_shape, activation='relu', ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3),
              input_shape=image_shape, activation='relu', ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

* **Summary of the model**

In [ ]:
create_tf_model().summary()

## Implementing early stopping to reduce risk of overfitting

* starting of with patience=6 and min_delta=0.005 (The minimum amount of change to count as sucessfull) 

In [32]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, mode="min")

### **Fit Model for Model training**

In [33]:
model = create_tf_model()
model.fit(train_set,
          epochs=25,
          steps_per_epoch=len(train_set.classes) // batch_size,
          validation_data=validation_set,
          callbacks=[early_stop],
          verbose=1
          )

Epoch 1/25
117/117 [==============================] - 159s 1s/step - loss: 0.3333 - accuracy: 0.8455 - val_loss: 0.1872 - val_accuracy: 0.9595
Epoch 2/25
117/117 [==============================] - 166s 1s/step - loss: 0.0762 - accuracy: 0.9781 - val_loss: 0.3171 - val_accuracy: 0.8690
Epoch 3/25
117/117 [==============================] - 151s 1s/step - loss: 0.0485 - accuracy: 0.9842 - val_loss: 0.0483 - val_accuracy: 0.9833
Epoch 4/25
117/117 [==============================] - 148s 1s/step - loss: 0.0322 - accuracy: 0.9908 - val_loss: 0.0544 - val_accuracy: 0.9905
Epoch 5/25
117/117 [==============================] - 150s 1s/step - loss: 0.0353 - accuracy: 0.9901 - val_loss: 0.0351 - val_accuracy: 0.9905
Epoch 6/25
117/117 [==============================] - 158s 1s/step - loss: 0.0408 - accuracy: 0.9880 - val_loss: 0.0042 - val_accuracy: 0.9976
Epoch 7/25
117/117 [==============================] - 155s 1s/step - loss: 0.0353 - accuracy: 0.9890 - val_loss: 0.0240 - val_accuracy: 0.9952

* Saving the model

In [34]:
model.save('output/v1/mildew_detection_model.h5')


---

# Push files to Repo

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
    # create here your folder
    # os.makedirs(name='')
except Exception as e:
    print(e)
